In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab and Kaggle notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 
dtype = None 
load_in_4bit = True 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/106k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",    
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

# Prepare dataset

In [ ]:
# preprocessing dataset
import pandas as pd
df_train = pd.read_csv("/content/train_translate.csv")

In [ ]:
df_train

,id,input,output
0,0,คึดฮอดเจ้าหลายเด้อ\n,คิดถึงเธอมากนะ\n
1,1,เจ้าสิไปเบิ่งคอนเสิร์ตหมอลำซิ่งนำกันบ่มื้อนี้แ...,คุณจะไปดูคอนเสิร์ตหมอลำซิ่งด้วยกันไหมเย็นนี้\n
2,2,เฮาสิไปเลาะในเมือง\n,เราจะไปเดินเล่นในเมือง\n
3,3,เฮามาเฮ็ดแนวกินนำกัน\n,เรามาทำอาหารด้วยกัน\n
4,4,เมื่อวานเฮาไปฟังหมอลำหน้าวัดม่วนหลายสนุกจนบ่มี...,เมื่อวานเราไปฟังหมอลำหน้าวัดสนุกมากจนไม่มีใครอ...
...,...,...,...
1938,1938,อันนี้แม่นบ่\n,อันนี้ใช่ไหม\n
1939,1939,สิไปหาซื้อของ\n,จะไปหาซื้อของ\n
1940,1940,เฮาเป็นหมู่กัน\n,เราเป็นเพื่อนกัน\n
1941,1941,สภาพจราจรมื้อนี้ติดหลาย\n,สภาพจราจรวันนี้ติดขัดมาก\n


In [ ]:
from datasets import Dataset

def create_dataset_from_df(df):
    """
    Convert DataFrame to Dataset format
    """
    dataset_dict = {
        'output': [],  # Column for target/output text
        'input': [],   # Column for input text
        'instruction': [],  # Column for instructions
    }

    for index, row in df.iterrows():
        # Add instruction for the task
        instruction = '''แปลภาษาไทยภาคอีสาน เป็นภาษาไทยภาคกลาง''' 

        # Add input text (the content to be analyzed)
        input_text = row['input'].replace("\n", "")

        # Add output (the evaluation/labels)
        output = row['output'].replace("\n", "")  


        # Append to dictionary
        dataset_dict['instruction'].append(instruction)
        dataset_dict['input'].append(input_text)
        dataset_dict['output'].append(output)

    # Create Dataset object
    dataset = Dataset.from_dict(dataset_dict)

    return dataset

# Create the dataset
dataset = create_dataset_from_df(df_train)

# Print information about the dataset
print(f"Dataset created with {len(dataset)} rows")
print("\nDataset features:", dataset.features)
print("\nSample from dataset:")
print(dataset[0])


Dataset created with 1943 rows

Dataset features: {'output': Value(dtype='string', id=None), 'input': Value(dtype='string', id=None), 'instruction': Value(dtype='string', id=None)}

Sample from dataset:
{'output': 'คิดถึงเธอมากนะ', 'input': 'คึดฮอดเจ้าหลายเด้อ', 'instruction': 'แปลภาษาไทยภาคอีสาน เป็นภาษาไทยภาคกลาง'}


In [ ]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/1943 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['output', 'input', 'instruction', 'text'],
    num_rows: 1943
})

In [ ]:


from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 3, # Set this for 1 full training run.
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/1943 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/1943 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/1943 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/1943 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,943 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 729
 "-____-"     Number of trainable parameters = 40,370,176


Step,Training Loss
1,1.723100
2,1.808800
3,1.693500
4,1.484000
5,1.320000
6,1.146800
7,1.154700
8,0.857600
9,0.730300
10,0.810600


# test

In [ ]:
import pandas as pd
test_data = pd.read_csv("/content/test (1).csv")
test_data.head()

,id,input
0,2055,ข่อยบ่มีเวลาหลาย\n
1,2056,บ่ฮู้ว่าสิใช้แอปนี้จั่งใด\n
2,2057,เจ้ามีเบอร์โทรข่อยบ่\n
3,2058,เฮามาสิลองเฮ็ดเบิ่ง\n
4,2059,ไปนอนก่อนเด้อ\n


In [ ]:
FastLanguageModel.for_inference(model) 
inputs = tokenizer(
[
    alpaca_prompt.format(
        '''แปลภาษาไทยภาคอีสาน เป็นภาษาไทยภาคกลาง''', # instruction
        test_data["input"][1].replace("\n", ""), # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
print((tokenizer.batch_decode(outputs)[0].split("### Response:\n")[1].strip())[:])
outputs

ไม่รู้ว่าจะใช้แอปนี้ยังไง<|endoftext|>


tensor([[ 38214,    374,    458,   7600,    429,  16555,    264,   3383,     11,
          34426,    448,    458,   1946,    429,   5707,   4623,   2266,     13,
           9645,    264,   2033,    429,  34901,  44595,    279,   1681,    382,
          14374,  29051,    510, 126097, 127382, 125451, 129298,  22929,  28319,
          35648, 123895, 126323, 123909, 127382, 125451, 129298, 129148,    271,
          14374,   5571,    510,  36142,  18625, 124451, 123959,  37213,  64741,
          35648,  30785, 124207,  19841, 128979,  54684,  20184, 124032,  60416,
          23271,  18625,  30434, 129442,    271,  14374,   5949,    510,  86032,
          18625,  22287, 123959,  37213,  64741, 123972, 124207,  19841, 128979,
          54684,  20184, 124032,  34509, 123874,  49756,  30434, 151643]],
       device='cuda:0')

In [ ]:
(tokenizer.batch_decode([[  271,  14374,   5949,    510,  86032,
          18625,  22287, 123959,  37213,  64741, 123972, 124207,  19841, 128979,
          54684,  20184, 124032,  34509, 123874,  49756,  30434, 151643]])[0])

'\n\n### Response:\nไม่รู้ว่าจะใช้แอปนี้ยังไง<|endoftext|>'

In [ ]:
def extract_sublist(tensor_data):
    # แปลง tensor ให้เป็น list (flatten ในกรณีที่เป็น multidimensional)
    data = tensor_data.flatten().tolist()
    # หา index ของค่า 271 ครั้งสุดท้ายใน list
    last_index = len(data) - 1 - data[::-1].index(271)
    # ดึง list ตั้งแต่ index ที่พบไปจนถึงจบ
    return data[last_index:]
result = extract_sublist(outputs)
result[4:-1]

[86032,
 18625,
 22287,
 123959,
 37213,
 64741,
 123972,
 124207,
 19841,
 128979,
 54684,
 20184,
 124032,
 34509,
 123874,
 49756,
 30434]

In [ ]:
print(tokenizer.batch_decode([result[4:-1]])[0])

ไม่รู้ว่าจะใช้แอปนี้ยังไง


In [ ]:
final_ans = []

for index, row in test_data.iterrows():
    print(row['input'])
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            '''แปลภาษาไทยภาคอีสาน เป็นภาษาไทยภาคกลาง''', # instruction
            row['input'].replace("\n", ""), # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True,temperature=0.15)
    result = extract_sublist(outputs)
    final_ans.append(tokenizer.batch_decode([result[4:-1]])[0])

ข่อยบ่มีเวลาหลาย

บ่ฮู้ว่าสิใช้แอปนี้จั่งใด

เจ้ามีเบอร์โทรข่อยบ่

เฮามาสิลองเฮ็ดเบิ่ง

ไปนอนก่อนเด้อ

เจ้าเป็นคนดีหลาย

อ้ายสิซื้อเสื้อใหม่

เจ้าสิบ่ไปเล่นน้ำบ่

ฮอดแล้วโทรบอกแหน่

มื้อนี่เฮ็ดแนวกินจังได๋

สิสมัครเรียนคอร์สใหม่อยู่บ่

ฝนตกอีกแล้วสิไปไสดี

อย่าซังข่อย

เฮาสิไปตลาดนัด

มื้อนี้เฮ็ดอีหยัง

เจ้าเป็นตาฮักหลาย

รถไฟฟ้าหยุดให้บริการยามใด๋

แม่นแล้วบ่

บ่ค่อยโอเคกับพฤติกรรมแบบนี้

สิไปทอดแหหาปลา

อยากได้คุณภาพดี

สินค้าโตนี้สามารถกันน้ำแล้วกะทนต่อแรงกระแทกได้หรือบ่

ขึ้นรถเร็วๆสิไปแล้ว

สิใส่น้ำแข็งหลายๆเด้อ

เจ้าสิบ่ไปเบิ่งคอนเสิร์ตบ่

เฮาสิมีประชุมมื้ออื่นเช้าจำไว้ให้ดีเด้ออย่าลืม

พายุเข้าอีสานฝนตกหนักหลายพื้นที่

จังได๋กะตาม

กินเข่าหรือยัง

สินค้านี้มีรับประกันจักปี

อาการดีขึ้นบ่

มื้ออื่นเฮาสิไปสวนสัตว์

เจ้าซังข่อยบ่

ร้านนี้รับบัตรเครดิตหรือเงินสด

ขอเปลี่ยนห้องได้บ่

เจ้าสิบ่ไปเฮ็ดงานบ่

อันนี้เป็นหยัง

ข่อยสิกลับบ้านก่อน

บริการดีหลาย

ข่อยสิพาไปเบิ่งหมอลำ

พ้อเชื้อโรคระบาดในฟาร์มหมู

สิไปเฮ็ดงาน

ฟ้าวมากินข้าวแหน่

ซ่อยแจ้งพนักงานให้แน

บ่เป็นหยังดอก

ฝนตกตลอดวัน



In [ ]:
# prompt: make final_ans join to test_data in column output

test_data['output'] = final_ans


In [ ]:
test_data

,id,input,output
0,2055,ข่อยบ่มีเวลาหลาย\n,ฉันไม่มีเวลาเลย
1,2056,บ่ฮู้ว่าสิใช้แอปนี้จั่งใด\n,ไม่รู้ว่าจะใช้แอปนี้ยังไง
2,2057,เจ้ามีเบอร์โทรข่อยบ่\n,เธอมีเบอร์โทรฉันไหม
3,2058,เฮามาสิลองเฮ็ดเบิ่ง\n,เรามาลองทำดู
4,2059,ไปนอนก่อนเด้อ\n,ไปนอนก่อนนะ
...,...,...,...
369,2424,กาแฟสิใช้เมล็ดจากบราซิลบ่\n,กาแฟใช้เมล็ดจากบราซิลไหม
370,2425,ถ้าซื้อหลายชิ้นลดเพิ่มได้บ่\n,ถ้าซื้อหลายชิ้นลดเพิ่มได้ไหม
371,2426,เจ้าคิดจังได๋\n,เธอคิดอะไรอยู่
372,2427,เฮ็ดเวียกอยู่บ้อ\n,ทำงานอยู่ไหม


In [ ]:
test_data.to_csv("test_translate_QWEN16.csv",index=False)

In [ ]:
answer = test_data[['id', 'output']]

In [ ]:
answer.to_csv("translate.csv",index=False)

In [ ]:
answer

,id,output
0,2055,ฉันไม่มีเวลาเลย
1,2056,ไม่รู้ว่าจะใช้แอปนี้ยังไง
2,2057,เธอมีเบอร์โทรฉันไหม
3,2058,เรามาลองทำดู
4,2059,ไปนอนก่อนนะ
...,...,...
369,2424,กาแฟใช้เมล็ดจากบราซิลไหม
370,2425,ถ้าซื้อหลายชิ้นลดเพิ่มได้ไหม
371,2426,เธอคิดอะไรอยู่
372,2427,ทำงานอยู่ไหม


In [ ]:
# เช็คคำตอบดูว่าเป็นยังไง
for i in range(400):
  print(i,answer["output"][i])

0 ฉันไม่มีเวลาเลย
1 ไม่รู้ว่าจะใช้แอปนี้ยังไง
2 เธอมีเบอร์โทรฉันไหม
3 เรามาลองทำดู
4 ไปนอนก่อนนะ
5 เธอเป็นคนดีมาก
6 พี่จะซื้อเสื้อตัวใหม่
7 เธอจะไม่ไปเล่นน้ำเหรอ
8 ถึงแล้วโทรบอกหน่อย
9 วันนี้ทำอะไรกินบ้าง
10 จะสมัครเรียนคอร์สใหม่อยู่ไหม
11 ฝนตกอีกแล้วจะไปไหนดี
12 อย่าเกลียดฉัน
13 เราจะไปตลาดนัด
14 วันนี้ทำอะไร
15 คุณเป็นตาฮักมาก
16 รถไฟฟ้าหยุดให้บริการเมื่อไหร่
17 ใช่แล้วหรือยัง
18 ไม่ค่อยโอเคกับพฤติกรรมแบบนี้
19 จะไปทอดย่างหาปลา
20 อยากได้คุณภาพดี
21 สินค้าตัวนี้สามารถกันน้ำและทนต่อแรงกระแทกได้หรือเปล่า
22 ขึ้นรถเร็วๆเดี๋ยวจะสาย
23 จะใส่น้ำแข็งเยอะๆนะ
24 เธอจะไม่ไปดูคอนเสิร์ตเหรอ
25 เราจะมีประชุมพรุ่งนี้เช้าจำไว้ให้ดีนะอย่าลืม
26 พายุเข้าอีสานฝนตกหนักมากในพื้นที่
27 ไม่รู้ว่าเป็นยังไง
28 กินข้าวหรือยัง
29 สินค้านี้มีรับประกันกี่ปี
30 อาการดีขึ้นไหม
31 พรุ่งนี้เราจะไปสวนสัตว์
32 เธอไม่ชอบฉันหรือว่าอย่างไร
33 ร้านนี้รับบัตรเครดิตหรือเงินสด
34 ขอเปลี่ยนห้องได้ไหม
35 เธอจะไม่ไปทำงานเหรอ
36 อันนี้คืออะไร
37 ฉันจะกลับบ้านก่อน
38 บริการดีมาก
39 ฉันจะพาไปดูหมอลำ
40 พบเชื้อโรคระบาดในฟาร์มหมู
4

KeyError: 374

In [ ]:
pip install pythainlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 82.2 MB/s eta 0:00:00


In [ ]:
from pythainlp.tokenize import word_tokenize

answer["output"] = answer["output"].apply(lambda text: " ".join(word_tokenize(text, engine="newmm")))

<ipython-input-28-ef1d53c8d3d4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer["output"] = answer["output"].apply(lambda text: " ".join(word_tokenize(text, engine="newmm")))


In [ ]:
answer.to_csv("translate2_no_post_process_QWENr16_GOD.csv",index=False)

In [ ]:
# translate2_no_post_process_QWENr16_GOD เอาอันนี้ส่ง 
# เนื่องจาก score private ที่ทำ post process กับไม่ทำคะแนนเท่ากันเลยคือ 90.8